# FINAL CAPSTONE PROJECT/ The battle of the neighborhoods

## Addressing a business problem using a combination of structured problem solving, data analysis & machine learning.

## CONTENTS
###  - Introduction
### - Data collection and cleaning
### - Methodology
### - Results section
###  - Discussion
### - Conclusion

## 1 Introduction/ Business Problem

### Paris is one of the most important cities in the world which attracts every year millions of tourists from all over the world. It is as well very famous for its excellent cuisine both national and international. One of the main actors in in international cousine is as well Italy which has exported all over the world its food culture starting from traditional Italian Pizza and spaghetti to many different regional food always appreciated.
### Is there still, room for Italian food in Paris? 
### We think yes in general because quality is always appreciated, but even more for some types of foods that, being very popular in Italy, even for strangers visiting the country, still have a limited presence (compared to others) in some countries like France and specifically in Paris. 
### We are looking in this case to home-made Ice-cream. Wea re looking for several high-traffic locations not necessary in the very centre of the town. So we focus on that borough during our analysis. We define potential neighbourhood based on the number of potential competitors for ice-cream, pastries and in general frozen sweet food which are operating right in each neighbourhood. Paris has full potential but also it is a very challenging district to open a business because of high competition. New shops should be open where we can ensure that we have enough customers, where we are not so close to direct competitors or where, if we are close to others, we are sure to provide the best quality in order to attract as many customers as possible.
### The scope of the project is to advise on the business strategies and execution roadmap on setting up an Italian Ice-cream shop in Paris. The initial business problem question is “where is then better location in Paris to open one (or more) Italian home-made Ice-cream shop?”.
### The City of Paris is divided for administrative reasons into 20 main boroughs (arrondissments) for administrative purposes. Each of these administrative districts (or arrondissements) are officially divided into 4 quartiers.
### You will find in this report a map showing Paris arrondissements and a map presenting Paris neighbourhoods. Within each of its boroughs, the neighbourhoods are providing the typical flavour of Paris with their own culture, locations and charme. The twenty arrondissements of Paris all have their own characteristics in terms of buildings, places to go out, restaurants, and in general aspects that can influence the success of an initiative to open a new activity.


## 2 Data Collecton and cleaning

### Paris has a total of 20 boroughs and 80 neighbourhoods. In order to segment the neighbourhoods and explore them, we will use sets of data available at data.govr.fr an open platform containing numerous sets of public French data.
### we will use 2 data sets:
### The first consists of the list of the 20 boroughs with their respective geographical coordinates:

### https://www.data.gouv.fr/fr/datasets/r/0d3553c6-45c0-4b16-82be-5ef314437d3e

### The second dataset consists of the 80 neighbourhoods with their respective geographical coordinates:

### https://www.data.gouv.fr/fr/datasets/r/a3b31fdc-85dc-4aeb-94c6-a8b57aebef77

### Merging these 2 datasets after having them cleaned we will have a final data frame with the 20 boroughs and the 4 neighbourhoods for each of them for a total of 80 with related geographicalcoordinates.
### We will then use Folium to represent all these locations on a Paris Map and the Foursquare API to provision venues information for each neighbourhood.

### With this additional information we will explore neighbourhood and we will cluster them in search of the best ones where to open our chain of home-made Ice cream.

## First we install all necessary libaries

In [1]:
! pip install folium
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


### We access the first file in repository Data.Gov.Fr, consisting of the list of the 20 boroughs with their respective geographical coordinates. we clean data and produce a simple dataframe with Borough Name and geographical data.

In [2]:
# Retrive url and transform in Pandas Dataframe
url ="https://www.data.gouv.fr/fr/datasets/r/0d3553c6-45c0-4b16-82be-5ef314437d3e"
df =pd.read_csv(url, sep=';')
df.head()

,n_sq_ar,c_ar,c_arinsee,l_ar,l_aroff,n_sq_co,surface,perimetre,geom_x_y,geom
0,750000003,3,75103,3ème Ardt,Temple,750001537,1.170883e+06,4519.263648,"48.86287238,2.3600009859","{""type"": ""Polygon"", ""coordinates"": [[[2.363828..."
1,750000011,11,75111,11ème Ardt,Popincourt,750001537,3.665442e+06,8282.011886,"48.8590592213,2.3800583082","{""type"": ""Polygon"", ""coordinates"": [[[2.396236..."
2,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","{""type"": ""Polygon"", ""coordinates"": [[[2.333806..."
3,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007..."
4,750000007,7,75107,7ème Ardt,Palais-Bourbon,750001537,4.090057e+06,8099.424883,"48.8561744288,2.31218769148","{""type"": ""Polygon"", ""coordinates"": [[[2.320902..."


In [3]:
# Reduce number of columns
df=df.drop(['n_sq_ar','n_sq_co','geom','surface','c_arinsee','l_ar','perimetre'], axis = 1)
df.head()

,c_ar,l_aroff,geom_x_y
0,3,Temple,"48.86287238,2.3600009859"
1,11,Popincourt,"48.8590592213,2.3800583082"
2,14,Observatoire,"48.8292445005,2.3265420442"
3,1,Louvre,"48.8625627018,2.33644336205"
4,7,Palais-Bourbon,"48.8561744288,2.31218769148"


In [4]:
# Rename columns
df = df.rename(columns={'c_ar': 'Borough_N','l_aroff': 'Borough', 'geom_x_y': 'Geo'})
df.head()

,Borough_N,Borough,Geo
0,3,Temple,"48.86287238,2.3600009859"
1,11,Popincourt,"48.8590592213,2.3800583082"
2,14,Observatoire,"48.8292445005,2.3265420442"
3,1,Louvre,"48.8625627018,2.33644336205"
4,7,Palais-Bourbon,"48.8561744288,2.31218769148"


In [5]:
# We split the column "Geo" into 2 dofferent columns "Lat" and "Long"
df["Lat"], df["long"] = df["Geo"].str.split(",", 2).str
df=df.drop(['Geo'], axis = 1)
df.head()

<ipython-input-5-642dedd87c12>:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df["Lat"], df["long"] = df["Geo"].str.split(",", 2).str


,Borough_N,Borough,Lat,long
0,3,Temple,48.86287238,2.3600009859
1,11,Popincourt,48.8590592213,2.3800583082
2,14,Observatoire,48.8292445005,2.3265420442
3,1,Louvre,48.8625627018,2.33644336205
4,7,Palais-Bourbon,48.8561744288,2.31218769148


### Ensure "Lat" and "Long" contain numeric data

In [6]:
df["Lat"] = pd.to_numeric(df["Lat"])
df["long"] = pd.to_numeric(df["long"])
df.head()

,Borough_N,Borough,Lat,long
0,3,Temple,48.862872,2.360001
1,11,Popincourt,48.859059,2.380058
2,14,Observatoire,48.829245,2.326542
3,1,Louvre,48.862563,2.336443
4,7,Palais-Bourbon,48.856174,2.312188


In [7]:
# Check data types
df.dtypes

Borough_N      int64
Borough       object
Lat          float64
long         float64
dtype: object

### We access the second file in repository Data.Gov.Fr, consisting of the list of the 20 neighboroughs with their respective geographical coordinates:we clean data and produce a simple dataframe with Neighborough Name and geographical data.

In [8]:
# access file and transform in Pandas dathttps://eu-gb.dataplatform.cloud.ibm.com/data/jupyter2/runtimeenv2/v1/wdpx/service/notebook/conda2py37oce38ef488a3fbe41c7b2536e1346e781f9/dsxjpy/KRzo_DilmGMY_-cPmHimqg:eejrmOZ_R4vH62ZXIbCN9RwhiexgFzIT5Y6YGwmKlp86W2CZSwqugSQmZtIB5oTBhCHO9LM/container/notebooks/0ed9a070-f2f6-424d-bcfe-344c32ee3ca7?api=v2&project=38ef488a-3fbe-41c7-b253-6e1346e781f9#We-access-the-second-file-in-repository-Data.Gov.Fr,-consisting-of-the-list-of-the-20-neighboroughs-with-their-respective-geographical-coordinates:we-clean-data-and-produce-a-simple-dataframe-with-Neighborough-Name-and-geographical-data.aframe
url ="https://www.data.gouv.fr/fr/datasets/r/a3b31fdc-85dc-4aeb-94c6-a8b57aebef77"
df1=pd.read_csv(url, sep=';')
df1.head()

,n_sq_qu,c_qu,c_quinsee,l_qu,c_ar,n_sq_ar,perimetre,surface,geom_x_y,geom
0,750000011,11,7510303,Archives,3,750000003,2534.100042,3.677284e+05,"48.8591924127,2.36320505733","{""type"": ""Polygon"", ""coordinates"": [[[2.368479..."
1,750000027,27,7510703,Ecole-Militaire,7,750000007,3887.703157,8.078708e+05,"48.8503592615,2.31103093627","{""type"": ""Polygon"", ""coordinates"": [[[2.320083..."
2,750000064,64,7511604,Chaillot,16,750000016,5207.046446,1.424035e+06,"48.8684336145,2.29167904274","{""type"": ""Polygon"", ""coordinates"": [[[2.292681..."
3,750000007,7,7510203,Mail,2,750000002,2179.153605,2.781426e+05,"48.8680083374,2.34469912743","{""type"": ""Polygon"", ""coordinates"": [[[2.346838..."
4,750000008,8,7510204,Bonne-Nouvelle,2,750000002,2233.976030,2.814482e+05,"48.8671501183,2.35008019041","{""type"": ""Polygon"", ""coordinates"": [[[2.351518..."


In [9]:
# Drop unnecessary columns
df1=df1.drop(['n_sq_ar','n_sq_qu','c_qu','c_quinsee','n_sq_ar','perimetre','surface','geom','perimetre'], axis = 1)
df1.head()

,l_qu,c_ar,geom_x_y
0,Archives,3,"48.8591924127,2.36320505733"
1,Ecole-Militaire,7,"48.8503592615,2.31103093627"
2,Chaillot,16,"48.8684336145,2.29167904274"
3,Mail,2,"48.8680083374,2.34469912743"
4,Bonne-Nouvelle,2,"48.8671501183,2.35008019041"


In [10]:
# rename columns
df1 = df1.rename(columns={'c_ar': 'Borough_N','l_qu': 'Neighborhood'})
df1.head()

,Neighborhood,Borough_N,geom_x_y
0,Archives,3,"48.8591924127,2.36320505733"
1,Ecole-Militaire,7,"48.8503592615,2.31103093627"
2,Chaillot,16,"48.8684336145,2.29167904274"
3,Mail,2,"48.8680083374,2.34469912743"
4,Bonne-Nouvelle,2,"48.8671501183,2.35008019041"


In [11]:
# We split the column "geom_x_y" into 2 dofferent columns "Latitude" and "Longitude"
df1["Latitude"], df1["Longitude"] = df1["geom_x_y"].str.split(",", 2).str
df1=df1.drop(['geom_x_y'], axis = 1)
df1.head()

<ipython-input-11-2dbd575e557d>:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df1["Latitude"], df1["Longitude"] = df1["geom_x_y"].str.split(",", 2).str


,Neighborhood,Borough_N,Latitude,Longitude
0,Archives,3,48.8591924127,2.36320505733
1,Ecole-Militaire,7,48.8503592615,2.31103093627
2,Chaillot,16,48.8684336145,2.29167904274
3,Mail,2,48.8680083374,2.34469912743
4,Bonne-Nouvelle,2,48.8671501183,2.35008019041


In [12]:
df1["Latitude"] = pd.to_numeric(df1["Latitude"])
df1["Longitude"] = pd.to_numeric(df1["Longitude"])
df.head()

,Borough_N,Borough,Lat,long
0,3,Temple,48.862872,2.360001
1,11,Popincourt,48.859059,2.380058
2,14,Observatoire,48.829245,2.326542
3,1,Louvre,48.862563,2.336443
4,7,Palais-Bourbon,48.856174,2.312188


# Tables merge

### We merge the 2 tables obtaiing a final One containing Boroughs, Neiborhoods and Geographical coordinates

In [13]:
# Merge based pon Borough_N
df_final= pd.merge(df,df1,on='Borough_N')
df_final=df_final.drop(['Lat','long'], axis = 1)
df_final.head()

,Borough_N,Borough,Neighborhood,Latitude,Longitude
0,3,Temple,Archives,48.859192,2.363205
1,3,Temple,Enfants-Rouges,48.863887,2.363123
2,3,Temple,Arts-et-Métiers,48.866470,2.357083
3,3,Temple,Sainte-Avoie,48.862557,2.354852
4,11,Popincourt,Saint-Ambroise,48.862345,2.376118


## Now we want to locate on map the Neighborhoods 

### Using Geolocator we find Paris geographical coordinates and using Folium we create a map of Paris

In [14]:
address = 'Paris, FR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [15]:
# create map of Paris using latitude and longitude values
map_Paris = folium.Map(location=[latitude, longitude], zoom_start=12)
map_Paris


### Always using Folium we add Neigborhoods mark ups on the map

In [16]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'],df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Paris)
map_Paris


## Now we are ready to use Foursquare to explore venues in our nighborhoods

###  We access foursquare

In [17]:
CLIENT_ID = 'J0LNJZNREBEIANXYPUP4X2KLVLUTRJMG35NKAXOLSMYH1LDM' # your Foursquare ID
CLIENT_SECRET = '4T5J2C4ZFSGYZUQFYT3OHVUE0GK41C2UB1CS3ZCRVU5HKFCR' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: J0LNJZNREBEIANXYPUP4X2KLVLUTRJMG35NKAXOLSMYH1LDM
CLIENT_SECRET:4T5J2C4ZFSGYZUQFYT3OHVUE0GK41C2UB1CS3ZCRVU5HKFCR


### We print the name and the coordinates of the first neighborhood in the table

In [18]:
neighborhood_latitude = df_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_final.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_final.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Archives are 48.859192412700004, 2.3632050573299996.


## Get the top 100 venues that are in Archives within a radius of 500 meters.

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '612774db4a709052a96c0dce'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Temple',
  'headerFullLocation': 'Temple, Paris',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 135,
  'suggestedBounds': {'ne': {'lat': 48.863692417200006,
    'lng': 2.370032116162571},
   'sw': {'lat': 48.8546924082, 'lng': 2.3563779984974285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0aa0eb009a0f47adbee9bf',
       'name': 'Galerie Thaddaeus Ropac',
       'location': {'address': '7 rue Debelleyme',
        'lat': 48.86058151476433,
        'lng': 2.3635389810078777,
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.8605

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

<ipython-input-21-22f06c73a3f4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Galerie Thaddaeus Ropac,Art Gallery,48.860582,2.363539
1,LECLAIREUR Sévigné,Clothing Store,48.857341,2.363273
2,Breizh Café,Creperie,48.860613,2.361804
3,Galerie Perrotin,Art Gallery,48.860726,2.365168
4,Musée Picasso,Art Museum,48.859905,2.362286
5,BrewDog Le Marais,Beer Bar,48.859201,2.362291
6,Chez Camille,French Restaurant,48.857782,2.360755
7,Musée Carnavalet,History Museum,48.857266,2.362267
8,SoMa,Japanese Restaurant,48.861511,2.362146
9,Aēsop,Cosmetics Shop,48.858998,2.359470


## Let's check the total number of venues by categories in the neighborhood Archives

In [22]:
nearby_venus_g=nearby_venues.groupby('categories').count()
#nearby_venus_s=nearby_venus_g.sort_values('name')
nearby_venus_s=nearby_venus_g.sort_values(by=['name'], ascending=False)
nearby_venus_s.head(10)

,name,lat,lng
categories,,,
French Restaurant,10,10,10
Hotel,6,6,6
Clothing Store,5,5,5
Bookstore,4,4,4
Art Gallery,3,3,3
Italian Restaurant,3,3,3
Bistro,3,3,3
Burger Joint,3,3,3
Plaza,3,3,3


In [23]:
nearby_venus_s.tail(10)

,name,lat,lng
categories,,,
Creperie,1,1,1
Israeli Restaurant,1,1,1
Ice Cream Shop,1,1,1
Dessert Shop,1,1,1
History Museum,1,1,1
Diner,1,1,1
Health Food Store,1,1,1
Art Museum,1,1,1
Gastropub,1,1,1


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Neighborhoods in Paris

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Write the code to run the above function on each neighborhood and create a new dataframe called Paris_venues.

In [26]:
Paris_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Archives
Enfants-Rouges
Arts-et-Métiers
Sainte-Avoie
Saint-Ambroise
Folie-Méricourt
Roquette
Sainte-Marguerite
Parc-de-Montsouris
Montparnasse
Petit-Montrouge
Plaisance
Place-Vendôme
Palais-Royal
Saint-Germain-l'Auxerrois
Halles
Ecole-Militaire
Invalides
Saint-Thomas-d'Aquin
Gros-Caillou
Europe
Faubourg-du-Roule
Champs-Elysées
Madeleine
Gare
Maison-Blanche
Croulebarbe
Salpêtrière
Ternes
Plaine de Monceaux
Batignolles
Epinettes
Père-Lachaise
Belleville
Charonne
Saint-Fargeau
Chaillot
Porte-Dauphine
Auteuil
Muette
Combat
Villette
Pont-de-Flandre
Amérique
Mail
Bonne-Nouvelle
Gaillon
Vivienne
Saint-Vincent-de-Paul
Porte-Saint-Martin
Porte-Saint-Denis
Hôpital-Saint-Louis
Grandes-Carrières
Goutte-d'Or
La Chapelle
Clignancourt
Grenelle
Saint-Lambert
Necker
Javel
Bel-Air
Quinze-Vingts
Picpus
Bercy
Sorbonne
Saint-Victor
Val-de-Grâce
Jardin-des-Plantes
Monnaie
Notre-Dame-des-Champs
Odéon
Saint-Germain-des-Prés
Saint-Georges
Faubourg-Montmartre
Rochechouart
Chaussée-d'Antin
Notre-Dame
Saint-Gerva

In [27]:
print(Paris_venues.shape)
Paris_venues.head()

(5356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Archives,48.859192,2.363205,Galerie Thaddaeus Ropac,48.860582,2.363539,Art Gallery
1,Archives,48.859192,2.363205,LECLAIREUR Sévigné,48.857341,2.363273,Clothing Store
2,Archives,48.859192,2.363205,Breizh Café,48.860613,2.361804,Creperie
3,Archives,48.859192,2.363205,Galerie Perrotin,48.860726,2.365168,Art Gallery
4,Archives,48.859192,2.363205,Musée Picasso,48.859905,2.362286,Art Museum


## Check how many venues were returned for each neighborhood

In [28]:
Paris_venues_tot=Paris_venues.groupby('Neighborhood').count()
Paris_venues_tot_s=Paris_venues_tot.sort_values(by=['Venue'], ascending=False)
Paris_venues_tot_s

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Vivienne,100,100,100,100,100,100
Chaussée-d'Antin,100,100,100,100,100,100
Archives,100,100,100,100,100,100
Gaillon,100,100,100,100,100,100
Folie-Méricourt,100,100,100,100,100,100
...,...,...,...,...,...,...
Saint-Fargeau,13,13,13,13,13,13
Porte-Dauphine,10,10,10,10,10,10
Muette,8,8,8,8,8,8


In [29]:
Paris_venues_tot_new= Paris_venues_tot_s[Paris_venues_tot_s['Venue'] > 80]
Paris_venues_tot_new.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Vivienne,100,100,100,100,100,100
Chaussée-d'Antin,100,100,100,100,100,100
Archives,100,100,100,100,100,100
Gaillon,100,100,100,100,100,100
Folie-Méricourt,100,100,100,100,100,100


In [30]:
print('There are {} uniques vanue categories.'.format(len(Paris_venues['Venue Category'].unique())))

There are 304 uniques vanue categories.


In [31]:
Paris_venues_tot_new=Paris_venues_tot_new.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category'], axis = 1)
Paris_venues_tot_new.head()

""
Neighborhood
Vivienne
Chaussée-d'Antin
Archives
Gaillon
Folie-Méricourt


In [32]:
print("The Venue Categories are", Paris_venues['Venue Category'].unique())

The Venue Categories are ['Art Gallery' 'Clothing Store' 'Creperie' 'Art Museum' 'Beer Bar'
 'French Restaurant' 'History Museum' 'Japanese Restaurant'
 'Cosmetics Shop' 'Gourmet Shop' 'Cocktail Bar' 'Okonomiyaki Restaurant'
 'Italian Restaurant' 'Bistro' 'Ice Cream Shop' 'Steakhouse' 'Café'
 'Sandwich Place' 'Bakery' 'Garden' 'Pizza Place' 'Deli / Bodega'
 'Korean Restaurant' 'Park' 'Falafel Restaurant' 'Hotel'
 'Israeli Restaurant' 'Health Food Store' 'Tea Room' 'Coffee Shop'
 'Library' 'Paper / Office Supplies Store' 'Farmers Market'
 'Tapas Restaurant' 'Plaza' 'Provençal Restaurant' 'Bookstore'
 'Burger Joint' 'Pastry Shop' 'Thai Restaurant' 'Wine Bar'
 'Asian Restaurant' 'Moroccan Restaurant' 'Historic Site' 'Gastropub'
 'Dessert Shop' 'Bar' 'Cajun / Creole Restaurant' 'Chocolate Shop' 'Diner'
 'Restaurant' 'Seafood Restaurant' 'Speakeasy' 'Brasserie' 'Supermarket'
 'Vietnamese Restaurant' 'Breakfast Spot' 'Latin American Restaurant'
 'Circus' 'Tattoo Parlor' 'Vegetarian / Vegan R

## Now we analyze each Neighborhood

## Create a new dataframe including only neigborhoods with more than 80 venues reported

In [33]:
Paris_venues = Paris_venues.merge(Paris_venues_tot_new, on='Neighborhood', how='inner')
#Paris_venues.to_excel(r'C:\Users\Paold\Export.xlsx', index = False)

Paris_venues.head()




,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Archives,48.859192,2.363205,Galerie Thaddaeus Ropac,48.860582,2.363539,Art Gallery
1,Archives,48.859192,2.363205,LECLAIREUR Sévigné,48.857341,2.363273,Clothing Store
2,Archives,48.859192,2.363205,Breizh Café,48.860613,2.361804,Creperie
3,Archives,48.859192,2.363205,Galerie Perrotin,48.860726,2.365168,Art Gallery
4,Archives,48.859192,2.363205,Musée Picasso,48.859905,2.362286,Art Museum


## We update DF_Final as well that will be needed for Clustering

In [34]:
df_final=df_final.merge(Paris_venues_tot_new, on='Neighborhood', how='inner')
df_final.head()

,Borough_N,Borough,Neighborhood,Latitude,Longitude
0,3,Temple,Archives,48.859192,2.363205
1,3,Temple,Enfants-Rouges,48.863887,2.363123
2,3,Temple,Arts-et-Métiers,48.866470,2.357083
3,3,Temple,Sainte-Avoie,48.862557,2.354852
4,11,Popincourt,Saint-Ambroise,48.862345,2.376118


In [35]:
# one hot encoding
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Neighborhood'] = Paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Paris_onehot.columns[-1]] + list(Paris_onehot.columns[:-1])
Paris_onehot = Paris_onehot[fixed_columns]
Paris_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Archives,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Archives,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Archives,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Archives,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Archives,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
Paris_onehot.shape

(3372, 250)

In [37]:
Paris_grouped = Paris_onehot.groupby('Neighborhood').mean().reset_index()
Paris_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Archives,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.03,...,0.000000,0.0,0.000000,0.0,0.00,0.010000,0.00,0.00,0.00,0.0
1,Arts-et-Métiers,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.02,...,0.000000,0.0,0.010000,0.0,0.03,0.050000,0.01,0.00,0.00,0.0
2,Batignolles,0.0,0.0,0.0,0.0,0.0,0.010101,0.0,0.00,0.00,...,0.020202,0.0,0.010101,0.0,0.00,0.010101,0.00,0.00,0.00,0.0
3,Bonne-Nouvelle,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,...,0.000000,0.0,0.010000,0.0,0.00,0.040000,0.00,0.02,0.00,0.0
4,Chaillot,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,...,0.000000,0.0,0.010000,0.0,0.01,0.010000,0.00,0.00,0.01,0.0


## confirm the new size

In [38]:
Paris_grouped.shape

(35, 250)

In [39]:
#Paris_grouped.to_excel(r'C:\Users\Paold\Export.xlsx', index = False)

## Let's filter venues categories more important for us using presence of theatres museums clothing stores and others as indication of traffic

In [95]:
Paris_grouped1= Paris_grouped[['Neighborhood','Clothing Store','Cosmetics Shop','Record Shop','Perfume Shop','Shopping Mall','Furniture / Home Store','Souvenir Shop','Garden','Ice Cream Shop']]
Paris_grouped1.head()

,Neighborhood,Clothing Store,Cosmetics Shop,Record Shop,Perfume Shop,Shopping Mall,Furniture / Home Store,Souvenir Shop,Garden,Ice Cream Shop
0,Archives,0.05,0.01,0.0,0.00,0.0,0.000000,0.0,0.02,0.01
1,Arts-et-Métiers,0.00,0.00,0.0,0.00,0.0,0.010000,0.0,0.00,0.00
2,Batignolles,0.00,0.00,0.0,0.00,0.0,0.010101,0.0,0.00,0.00
3,Bonne-Nouvelle,0.02,0.01,0.0,0.01,0.0,0.000000,0.0,0.00,0.01
4,Chaillot,0.00,0.01,0.0,0.00,0.0,0.000000,0.0,0.00,0.00


In [96]:
#Paris_grouped1.to_excel(r'C:\Users\Paold\Export.xlsx', index = False)
#Paris_grouped1.count('Ice Cream Shop')

## Print each neighborhood along with the top 5 most common venues

In [97]:
num_top_venues = 5

for hood in Paris_grouped1['Neighborhood']:
    print("----"+hood+"----")
    temp = Paris_grouped1[Paris_grouped1['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Archives----
            venue  freq
0  Clothing Store  0.05
1          Garden  0.02
2  Cosmetics Shop  0.01
3  Ice Cream Shop  0.01
4     Record Shop  0.00


----Arts-et-Métiers----
                    venue  freq
0  Furniture / Home Store  0.01
1          Clothing Store  0.00
2          Cosmetics Shop  0.00
3             Record Shop  0.00
4            Perfume Shop  0.00


----Batignolles----
                    venue  freq
0  Furniture / Home Store  0.01
1          Clothing Store  0.00
2          Cosmetics Shop  0.00
3             Record Shop  0.00
4            Perfume Shop  0.00


----Bonne-Nouvelle----
            venue  freq
0  Clothing Store  0.02
1  Cosmetics Shop  0.01
2    Perfume Shop  0.01
3  Ice Cream Shop  0.01
4     Record Shop  0.00


----Chaillot----
            venue  freq
0  Cosmetics Shop  0.01
1  Clothing Store  0.00
2     Record Shop  0.00
3    Perfume Shop  0.00
4   Shopping Mall  0.00


----Champs-Elysées----
                    venue  freq
0          Clothin

## Put that into a pandas dataframe

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Paris_grouped1['Neighborhood']

for ind in np.arange(Paris_grouped1.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Paris_grouped1.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Archives,Clothing Store,Garden,Ice Cream Shop,Cosmetics Shop,Souvenir Shop,Furniture / Home Store,Shopping Mall
1,Arts-et-Métiers,Furniture / Home Store,Ice Cream Shop,Garden,Souvenir Shop,Shopping Mall,Perfume Shop,Record Shop
2,Batignolles,Furniture / Home Store,Ice Cream Shop,Garden,Souvenir Shop,Shopping Mall,Perfume Shop,Record Shop
3,Bonne-Nouvelle,Clothing Store,Ice Cream Shop,Perfume Shop,Cosmetics Shop,Garden,Souvenir Shop,Furniture / Home Store
4,Chaillot,Cosmetics Shop,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop


In [100]:
# get counts of Clothing Store in each Neighborhood


## Cluster Neighborhoods

In [101]:
kclusters = 4
Paris_grouped_clustering = Paris_grouped1.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_grouped_clustering)
kmeans.labels_[0:10]

array([0, 2, 2, 2, 2, 0, 0, 2, 2, 0])

In [102]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Paris_merged = df_final

#merge Paris_grouped with Paris_data to add latitude/longitude for each neighborhood
Paris_merged = Paris_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Paris_merged.head() # check the last columns!

,Borough_N,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,3,Temple,Archives,48.859192,2.363205,0,Clothing Store,Garden,Ice Cream Shop,Cosmetics Shop,Souvenir Shop,Furniture / Home Store,Shopping Mall
1,3,Temple,Enfants-Rouges,48.863887,2.363123,2,Clothing Store,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop
2,3,Temple,Arts-et-Métiers,48.866470,2.357083,2,Furniture / Home Store,Ice Cream Shop,Garden,Souvenir Shop,Shopping Mall,Perfume Shop,Record Shop
3,3,Temple,Sainte-Avoie,48.862557,2.354852,2,Garden,Ice Cream Shop,Souvenir Shop,Furniture / Home Store,Cosmetics Shop,Clothing Store,Shopping Mall
4,11,Popincourt,Saint-Ambroise,48.862345,2.376118,2,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop


In [103]:
Paris_merged["Cluster Labels"] = pd.to_numeric(Paris_merged["Cluster Labels"])
Paris_merged['Cluster Labels'] = Paris_merged['Cluster Labels'].astype(int)
Paris_merged.head()


,Borough_N,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,3,Temple,Archives,48.859192,2.363205,0,Clothing Store,Garden,Ice Cream Shop,Cosmetics Shop,Souvenir Shop,Furniture / Home Store,Shopping Mall
1,3,Temple,Enfants-Rouges,48.863887,2.363123,2,Clothing Store,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop
2,3,Temple,Arts-et-Métiers,48.866470,2.357083,2,Furniture / Home Store,Ice Cream Shop,Garden,Souvenir Shop,Shopping Mall,Perfume Shop,Record Shop
3,3,Temple,Sainte-Avoie,48.862557,2.354852,2,Garden,Ice Cream Shop,Souvenir Shop,Furniture / Home Store,Cosmetics Shop,Clothing Store,Shopping Mall
4,11,Popincourt,Saint-Ambroise,48.862345,2.376118,2,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop


In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['Neighborhood'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters
       

## Examine clusters

## Cluster 1

In [105]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 0, Paris_merged.columns[[1,2] + list(range(5, Paris_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Temple,Archives,0,Clothing Store,Garden,Ice Cream Shop,Cosmetics Shop,Souvenir Shop,Furniture / Home Store,Shopping Mall
12,Élysée,Faubourg-du-Roule,0,Cosmetics Shop,Clothing Store,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall
13,Élysée,Champs-Elysées,0,Garden,Clothing Store,Cosmetics Shop,Furniture / Home Store,Ice Cream Shop,Souvenir Shop,Shopping Mall
14,Élysée,Madeleine,0,Clothing Store,Garden,Furniture / Home Store,Shopping Mall,Cosmetics Shop,Ice Cream Shop,Souvenir Shop
31,Opéra,Chaussée-d'Antin,0,Clothing Store,Garden,Souvenir Shop,Cosmetics Shop,Ice Cream Shop,Furniture / Home Store,Shopping Mall


## Cluster 2

In [106]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 1, Paris_merged.columns[[1,2] + list(range(5, Paris_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
11,Palais-Bourbon,Gros-Caillou,1,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop
24,Panthéon,Sorbonne,1,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop
26,Luxembourg,Monnaie,1,Ice Cream Shop,Garden,Souvenir Shop,Cosmetics Shop,Clothing Store,Furniture / Home Store,Shopping Mall
28,Luxembourg,Saint-Germain-des-Prés,1,Ice Cream Shop,Clothing Store,Souvenir Shop,Cosmetics Shop,Garden,Furniture / Home Store,Shopping Mall
32,Hôtel-de-Ville,Notre-Dame,1,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop


## Cluster 3

In [107]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 2, Paris_merged.columns[[1,2] + list(range(5, Paris_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,Temple,Enfants-Rouges,2,Clothing Store,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop
2,Temple,Arts-et-Métiers,2,Furniture / Home Store,Ice Cream Shop,Garden,Souvenir Shop,Shopping Mall,Perfume Shop,Record Shop
3,Temple,Sainte-Avoie,2,Garden,Ice Cream Shop,Souvenir Shop,Furniture / Home Store,Cosmetics Shop,Clothing Store,Shopping Mall
4,Popincourt,Saint-Ambroise,2,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop
5,Popincourt,Folie-Méricourt,2,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop
6,Popincourt,Roquette,2,Record Shop,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop
7,Observatoire,Montparnasse,2,Ice Cream Shop,Garden,Souvenir Shop,Furniture / Home Store,Shopping Mall,Perfume Shop,Record Shop
8,Louvre,Place-Vendôme,2,Garden,Perfume Shop,Cosmetics Shop,Clothing Store,Ice Cream Shop,Souvenir Shop,Furniture / Home Store
9,Louvre,Palais-Royal,2,Garden,Furniture / Home Store,Shopping Mall,Perfume Shop,Cosmetics Shop,Ice Cream Shop,Souvenir Shop
10,Palais-Bourbon,Saint-Thomas-d'Aquin,2,Garden,Ice Cream Shop,Furniture / Home Store,Clothing Store,Souvenir Shop,Shopping Mall,Perfume Shop


## Cluster 4

In [108]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 3, Paris_merged.columns[[1,2] + list(range(5, Paris_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
33,Hôtel-de-Ville,Saint-Gervais,3,Ice Cream Shop,Clothing Store,Garden,Furniture / Home Store,Perfume Shop,Cosmetics Shop,Souvenir Shop
34,Hôtel-de-Ville,Saint-Merri,3,Ice Cream Shop,Garden,Clothing Store,Souvenir Shop,Furniture / Home Store,Cosmetics Shop,Shopping Mall


In [109]:
#Paris_merged.loc[Paris_merged['Cluster Labels'] == 4, Paris_merged.columns[[1,2] + list(range(5, Paris_merged.shape[1]))]]


##  We exclude immediately Cluster 2 and 4 where existing Ice cream shops are dominating all the categories of shops indicating traffic.

## As an example in Saint-Gervais in cluster 4 we have 5 ice cream shops which is the second category after French Restaurant.

In [92]:
Paris_venues_N= Paris_venues[Paris_venues['Neighborhood'] == 'Saint-Gervais']

In [91]:
Paris_venues_FIN=Paris_venues_N.groupby('Venue Category').count()
Paris_venues_SORT=Paris_venues_FIN.sort_values(by=['Venue'], ascending=False)
Paris_venues_SORT.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
French Restaurant,11,11,11,11,11,11
Ice Cream Shop,5,5,5,5,5,5
Clothing Store,5,5,5,5,5,5
Hotel,4,4,4,4,4,4
Pastry Shop,4,4,4,4,4,4


## We exclude as well Cluster 3 and We decide to focus on the Cluster Number 1 for further Analysis

## Let' analyize Fisrt neigborhood Archives	

In [57]:
Paris_venues_N= Paris_venues[Paris_venues['Neighborhood'] == 'Archives']
Paris_venues_FIN=Paris_venues_N.groupby('Venue Category').count()
Paris_venues_SORT=Paris_venues_FIN.sort_values(by=['Venue'], ascending=False)
Paris_venues_SORT

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
French Restaurant,10,10,10,10,10,10
Hotel,6,6,6,6,6,6
Clothing Store,5,5,5,5,5,5
Bookstore,4,4,4,4,4,4
Art Gallery,3,3,3,3,3,3
Italian Restaurant,3,3,3,3,3,3
Bistro,3,3,3,3,3,3
Burger Joint,3,3,3,3,3,3
Plaza,3,3,3,3,3,3


# Archives looks a good potential for the scope of this analysis since it contais several venues indicating an high level of traffic only ONE ice cream Shop

# Same for Faubourg-du-Roul with a lot of venues indicating high traffic and one only ice creal shop

In [94]:
Paris_venues_N= Paris_venues[Paris_venues['Neighborhood'] == 'Faubourg-du-Roule']
Paris_venues_FIN=Paris_venues_N.groupby('Venue Category').count()
Paris_venues_SORT=Paris_venues_FIN.sort_values(by=['Venue'], ascending=False)
Paris_venues_SORT

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Hotel,15,15,15,15,15,15
French Restaurant,13,13,13,13,13,13
Jewelry Store,5,5,5,5,5,5
Cosmetics Shop,5,5,5,5,5,5
Café,4,4,4,4,4,4
Clothing Store,3,3,3,3,3,3
Japanese Restaurant,3,3,3,3,3,3
Sandwich Place,3,3,3,3,3,3
Department Store,2,2,2,2,2,2


# let' check now Madeleine

In [111]:
Paris_venues_N= Paris_venues[Paris_venues['Neighborhood'] == 'Madeleine']
Paris_venues_FIN=Paris_venues_N.groupby('Venue Category').count()
Paris_venues_SORT=Paris_venues_FIN.sort_values(by=['Venue'], ascending=False)
Paris_venues_SORT

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Hotel,14,14,14,14,14,14
French Restaurant,11,11,11,11,11,11
Boutique,9,9,9,9,9,9
Women's Store,4,4,4,4,4,4
Gourmet Shop,4,4,4,4,4,4
Dessert Shop,3,3,3,3,3,3
Clothing Store,3,3,3,3,3,3
Sandwich Place,2,2,2,2,2,2
Men's Store,2,2,2,2,2,2


## Agian a potential good place with many vanues indicating potential traffic and No Ice cream shops aleady opened.

# RESULTS

## After clustering the data of the respective neighborhoods, and analyzing data, we have found several Neighborhoods in cluster 4 where it looks valuable to open an Italian Ice cream Shop like Archives, Enfants-Rouges Madeleine, Champs-Elysée.

# OBSERVATION AND RECCOMENDATION

## The analysis should be continued to extract a certain number of locations and the final selection should be done and validated using additional data like the price per square meter located which gives as well the idea of the cost related to the opening of a shop in the specific Area.
## The copleteness of Foursquare data for Paris is progressing but not yet at the level uf US, Canada or certain countries in Far east.

# CONCLUSION

## Destpite it can be improved, the analysis clearly shows that there is room for Italian Ice cram shops in Paris in areas where the level of traffic is more than sufficient to ensure a goog level of profitability.
